In [7]:
# To work with local eumap code
# import sys
# sys.path.append('../../')

import os
from pathlib import Path
import pandas as pd
import geopandas as gpd
from eumap.mapper import LandMapper

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.8/site-packages/eumap/misc.py:17: UserWarning: ERROR: No module named 'gspread'

Encountered because misc.GoogleSheet has additional dependencies, please try:

	pip install eumap[full]

  warnings.warn(


In [8]:
from sklearn.metrics import log_loss

def log_loss_scorer(clf, X, y_true):
    class_labels = clf.classes_
    y_pred_proba = clf.predict_proba(X)
    error = log_loss(y_true, y_pred_proba, labels=class_labels)
    return error * -1

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(n_estimators=100)

hyperpar = GridSearchCV(
    estimator = estimator,
    scoring = log_loss_scorer,
    param_grid = {
     'min_samples_leaf': [1, 5],
     'max_depth': [5, None],
     'max_features': [0.5]
    }
)


In [12]:
root_path = '/home/opengeohub/faen/Desktop/OpenGeoHub2023_Hackathon_Tickbites_dummy'

training data preprocessing

In [13]:
tick_dummy = gpd.read_file(os.path.join(root_path,'tick_reports_dummy.gpkg'))

In [14]:
tick_dummy.head()

,id,acc,datetime,X,Y,geometry
0,48,74,2015-03-11 21:50:30.846000+00:00,2.703944e+06,1.233372e+06,POINT (2703943.983 1233372.237)
1,52,73,2015-03-12 00:02:37.652000+00:00,2.700859e+06,1.233586e+06,POINT (2700859.445 1233586.319)
2,59,257,2015-03-13 09:51:16.208000+00:00,2.559004e+06,1.153785e+06,POINT (2559003.820 1153784.800)
3,63,190,2015-03-14 17:42:58.027000+00:00,2.604691e+06,1.200648e+06,POINT (2604690.901 1200648.124)
4,66,333,2015-03-15 21:51:33.101000+00:00,2.681582e+06,1.247041e+06,POINT (2681581.972 1247040.561)


In [15]:
tick_dummy['date'] = tick_dummy['datetime'].apply(lambda x: x.strftime('%Y-%m'))
tick_dummy['month'] = tick_dummy['datetime'].apply(lambda x: x.strftime('%m'))
tick_dummy['year'] = tick_dummy['datetime'].apply(lambda x: x.strftime('%Y'))
tick_dummy = tick_dummy[['id','acc','geometry','date','year']]

In [16]:
tick_dummy 

,id,acc,geometry,date,year
0,48,74,POINT (2703943.983 1233372.237),2015-03,2015
1,52,73,POINT (2700859.445 1233586.319),2015-03,2015
2,59,257,POINT (2559003.820 1153784.800),2015-03,2015
3,63,190,POINT (2604690.901 1200648.124),2015-03,2015
4,66,333,POINT (2681581.972 1247040.561),2015-03,2015
...,...,...,...,...,...
28441,49910,272,POINT (2710148.950 1092354.692),2020-06,2020
28442,49949,269,POINT (2693209.436 1254548.143),2020-07,2020
28443,56372,969,POINT (2651868.645 1268156.188),2018-09,2018
28444,58623,301,POINT (2628845.592 1236824.861),2016-08,2016


In [367]:
static_fn_layers = []
#for i in os.scandir('/home/opengeohub/faen/Desktop/OpenGeoHub2023_Hackathon_Tickbites_dummy/prepared_2/'):
#    for j in os.scandir(i.path):
#        if j.name.endswith('.tif'):
#            static_fn_layers.append(Path(j.path))

In [368]:
for i in os.scandir(os.join.path(root_path,'external_input'):
    if i.name == 'static':
        for j in os.scandir(i.path):
            static_fn_layers.append(Path(j.path))

In [369]:
from eumap.mapper import SpaceOverlay

spc_overlay = SpaceOverlay(os.path.join(root_path,'training_points.gpkg'), fn_layers=static_fn_layers)

result = spc_overlay.run()


print(result.shape)

[14:45:56] 1/11 clm_lst_mod11a2.nighttime.trend.logit.ols.beta_m_1km_s0..0cm_2000..2021_mood_v1.2
[14:45:56] 2/11 dtm_twi_merit.dem_m_1km_s0..0cm_2017_mood_v1
[14:45:56] 3/11 clm_lst_mod11a2.nighttime.trend.logit.ols.alpha_m_1km_s0..0cm_2000..2021_mood_v1.2
[14:45:56] 4/11 dtm_floodmap.500y_jrc.hazardmapping_m_1km_s0..0cm_1500..2016_mood_v1.0
[14:45:56] 5/11 dtm_elevation_glo90.copernicus_m_1km_s0..0cm_2019_epsg.4326_mood_v1.0
[14:45:57] 6/11 dtm_slope_merit.dem_m_1km_s0..0cm_2017_mood_v1
[14:45:57] 7/11 dtm_height.above_meritdem.hydro_m_1km_s0..0cm_2020_mood_v1
[14:45:57] 8/11 dtm_northness_geomorphon90m_m_1km_s0..0cm_2018_mood_v1
[14:45:57] 9/11 dtm_upstream.area_meritdem.hydro_m_1km_s0..0cm_2020_mood_v1
[14:45:57] 10/11 dtm_slope.pct_geomorphon90m_m_1km_s0..0cm_2018_mood_v1
[14:45:57] 11/11 dtm_eastness_geomorphon90m_m_1km_s0..0cm_2018_mood_v1
(7594, 17)


In [370]:
result.head()

,id,acc,date,year,geometry,overlay_id,clm_lst_mod11a2.nighttime.trend.logit.ols.beta_m_1km_s0..0cm_2000..2021_mood_v1.2,dtm_twi_merit.dem_m_1km_s0..0cm_2017_mood_v1,clm_lst_mod11a2.nighttime.trend.logit.ols.alpha_m_1km_s0..0cm_2000..2021_mood_v1.2,dtm_floodmap.500y_jrc.hazardmapping_m_1km_s0..0cm_1500..2016_mood_v1.0,dtm_elevation_glo90.copernicus_m_1km_s0..0cm_2019_epsg.4326_mood_v1.0,dtm_slope_merit.dem_m_1km_s0..0cm_2017_mood_v1,dtm_height.above_meritdem.hydro_m_1km_s0..0cm_2020_mood_v1,dtm_northness_geomorphon90m_m_1km_s0..0cm_2018_mood_v1,dtm_upstream.area_meritdem.hydro_m_1km_s0..0cm_2020_mood_v1,dtm_slope.pct_geomorphon90m_m_1km_s0..0cm_2018_mood_v1,dtm_eastness_geomorphon90m_m_1km_s0..0cm_2018_mood_v1
0,245,134,2015-05,2015,POINT (2761122.335 1199736.378),1,93.0,96.0,8289.0,0.0,564.0,11.0,28.173571,0.016289,97.191231,6.518714,0.037920
1,246,175,2015-05,2015,POINT (2760869.989 1199016.156),2,89.0,91.0,8292.0,0.0,575.0,15.0,35.359009,0.024830,84.473320,8.315491,0.077564
2,250,301,2015-05,2015,POINT (2679020.257 1242855.502),3,59.0,98.0,8359.0,0.0,635.0,3.0,91.082863,-0.002399,0.525954,12.371400,-0.023437
3,251,617,2015-05,2015,POINT (2614113.641 1177744.489),4,86.0,105.0,8298.0,0.0,570.0,3.0,9.112237,-0.004365,52.942444,1.397063,-0.012183
4,252,617,2015-05,2015,POINT (2574821.229 1186701.631),5,73.0,109.0,8271.0,0.0,623.0,2.0,37.097443,-0.011198,0.543461,4.915729,0.009041


In [371]:
all_year = pd.DataFrame()
for year in ['2015','2016','2017','2018','2019','2020']:
    fn_layers = []
    for i in os.scandir(os.path.join(root_path,'external_input'):
        if i.name == year:
            for j in os.scandir(i.path):
                fn_layers.append(Path(j.path))
    #for i in os.scandir('/home/opengeohub/faen/Desktop/OpenGeoHub2023_Hackathon_Tickbites_dummy/prepared_2/Weather/'):
    #    for j in os.scandir(i.path):
    #        if j.name.split('_')[1].split('-')[0] == year and j.name.endswith('tif'):
    #            fn_layers.append(Path(j.path))
    spc_overlay = SpaceOverlay(os.path.join(root_path,'training_points.gpkg', fn_layers=fn_layers)

    result_tmp = spc_overlay.run()
    result_tmp = result_tmp[result_tmp.year==year]
    result_tmp.columns = result_tmp.columns.str.replace(year, '')
    all_year = pd.concat([all_year,result_tmp])

[14:46:02] 1/2 clm_precipitation_chelsea.annual_sum_1km_s0..0cm_2015_mood_v2.1
[14:46:02] 2/2 adm_population.density_gpw_m_1km_s0..0cm_2015_mood_v4
[14:46:03] 1/2 clm_precipitation_chelsea.annual_sum_1km_s0..0cm_2016_mood_v2.1
[14:46:03] 2/2 adm_population.density_gpw_m_1km_s0..0cm_2016_mood_v4
[14:46:03] 1/2 clm_precipitation_chelsea.annual_sum_1km_s0..0cm_2017_mood_v2.1
[14:46:03] 2/2 adm_population.density_gpw_m_1km_s0..0cm_2017_mood_v4
[14:46:04] 1/2 clm_precipitation_chelsea.annual_sum_1km_s0..0cm_2018_mood_v2.1
[14:46:04] 2/2 adm_population.density_gpw_m_1km_s0..0cm_2018_mood_v4
[14:46:04] 1/2 clm_precipitation_chelsea.annual_sum_1km_s0..0cm_2019_mood_v2.1
[14:46:04] 2/2 adm_population.density_gpw_m_1km_s0..0cm_2019_mood_v4
[14:46:05] 1/2 clm_precipitation_chelsea.annual_sum_1km_s0..0cm_2020_mood_v2.1
[14:46:05] 2/2 adm_population.density_gpw_m_1km_s0..0cm_2020_mood_v4


In [372]:
df = result.join(all_year, lsuffix='', rsuffix='_other')

In [373]:
columns = []
for i in df.columns:
    if i == 'Population_2015-2021':
        continue
    if not i.endswith('other'):
        columns.append(i)

In [376]:
feat_col_prfxs = list(columns[6:10])
target_col = 'acc'

min_samples_per_class = 0.05
cv = 5

landmapper_prob = LandMapper(points=df,
                        feat_col_prfxs = feat_col_prfxs,
                        target_col = target_col,
                        estimator = estimator,
                        hyperpar_selection = hyperpar,
                        cv = cv,
                        min_samples_per_class=min_samples_per_class,
                        pred_method='predict_proba',
                        verbose = True)

[14:46:17] Removing 6146 samples (acc in [134 175 617  41 533 333  18  91 265  52 797  16 168 514  87  77  29  65
  72  47 743  32 382 203 190 290 366 369  19  17  42  67 702 211 128 409
  76 320 176  21 183 700 494 519 492 184 287 150 156 297 852 102 453 317
 907 609 125 274 304  38 120 263  45 246 405 992 447 459 590  40  86 352
 425   8  62 358 160  12 332 331 508 451 586 698 972 132 108 555  61 878
 337 196 122 696  57 245 487 902  43 898 222 159 363  89  31  99  34 767
  33  54 396 326 251 237 323 845 381  37 697 406 233 115 815  82 192 778
  93 112 851 540 378 432 127 129  84 186 207 202 688 779  96  70 738 278
 294 354 574 123 777 167 219 153 292  95 390 526 226  48 189  24  71 113
 393 865 295 339  85 560  56 786 564 655 718 187  83  27 619 874  97 271
 308  69 193 137  22 603 303 119 244 800 200 279 653 147  75 750 908 806
 456 476 441 638 126 607 923 166 842 847 281 298 798 307 235 645 517 444
 814 253 430 969 111  88 753 228 813 124 296 169 864  64 170 157 692 252
  74 254 3

In [377]:
landmapper_prob.target_le

LabelEncoder()

In [378]:
landmapper_prob.train()

[14:46:19] Optimizing hyperparameters for RandomForestClassifier
[14:46:23]  0.64339 (+/-0.02575) from {'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 1}
[14:46:23]  0.64347 (+/-0.02950) from {'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 5}
[14:46:23]  0.71711 (+/-0.15268) from {'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 1}
[14:46:23]  0.65892 (+/-0.06085) from {'max_depth': None, 'max_features': 0.5, 'min_samples_leaf': 5}
[14:46:23] Best: -0.64339 using {'max_depth': 5, 'max_features': 0.5, 'min_samples_leaf': 1}
[14:46:23] Calculating evaluation metrics


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[14:46:24] Training RandomForestClassifier using all samples


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.9s finished


In [379]:
print(f'Log loss: {landmapper_prob.eval_metrics["log_loss"]:.3f}\n\n')
print(landmapper_prob.eval_report)

Log loss: 0.642


     log_loss   pr_auc   optimal_prob  optimal_precision  optimal_recall  support

0  1.0944     0.3597   0.3662        0.3566              0.3566         488
1  0.4127     0.6840   0.6340        0.6729              0.6729         960

Total                                                                     1448



In [381]:
fns_layers = static_fn_layers+fn_layers

In [383]:

fn_output = os.path.join(root_path, 'tick_acc.tif')

output_fn_files = landmapper_prob.predict(fn_layers=fns_layers, fn_output=fn_output, allow_additional_layers=True)

print('Output files:')
for output_fn_file in output_fn_files:
    print(f' - {Path(output_fn_file).name}')

[14:46:47] Reading 13 raster files using 4 workers
[14:46:50] Executing RandomForestClassifier
[14:47:36] RandomForestClassifier prediction time: 45.77 segs
Output files:
 - tick_acc_b1.tif
 - tick_acc_b2.tif
 - tick_acc_hcl.tif
 - tick_acc_hcl_prob.tif


/opt/conda/lib/python3.8/site-packages/eumap/misc.py:17: UserWarning: ERROR: No module named 'gspread'

Encountered because misc.GoogleSheet has additional dependencies, please try:

	pip install eumap[full]

  warnings.warn(
/opt/conda/lib/python3.8/site-packages/eumap/misc.py:17: UserWarning: ERROR: No module named 'gspread'

Encountered because misc.GoogleSheet has additional dependencies, please try:

	pip install eumap[full]

  warnings.warn(
/opt/conda/lib/python3.8/site-packages/eumap/misc.py:17: UserWarning: ERROR: No module named 'gspread'

Encountered because misc.GoogleSheet has additional dependencies, please try:

	pip install eumap[full]

  warnings.warn(
